In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
df = pd.read_csv(r'D:\QUANT A MAZE\cancerdata.csv')
print(df.head())
df = df.drop(columns=['id'])
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})
features = df.drop(columns=['diagnosis'])
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)
scaled_data = pd.DataFrame(scaled_features, columns=features.columns)
scaled_data['diagnosis'] = df['diagnosis']
print(scaled_data.head())

         id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842302         M        17.99         10.38          122.80     1001.0   
1    842517         M        20.57         17.77          132.90     1326.0   
2  84300903         M        19.69         21.25          130.00     1203.0   
3  84348301         M        11.42         20.38           77.58      386.1   
4  84358402         M        20.29         14.34          135.10     1297.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   
3          0.14250           0.28390          0.2414              0.10520   
4          0.10030           0.13280          0.1980              0.10430   

   ...  radius_se  texture_se  perimeter_se  area_se  smoothne

In [1]:
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_aer import AerSimulator
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
import numpy as np

In [2]:
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit_aer import AerSimulator
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
import numpy as np

In [ ]:
var_form = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz', reps=3)
optimizer = L_BFGS_B(maxiter=100)
simulator = AerSimulator()
hamiltonian = SparsePauliOp.from_list([("Z", 1), ("I", 1)])
estimator = Estimator()
vqe = VQE(estimator=estimator, ansatz=var_form, optimizer=optimizer)
result = vqe.compute_minimum_eigenvalue(hamiltonian)

C:\Users\MEGANATH N\AppData\Local\Temp\ipykernel_15384\2717697544.py:14: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


In [ ]:
optimal_point = result.optimal_point
optimal_value = result.optimal_value
optimizer_evals = result.optimizer_evals
optimizer_result = result.optimizer_result
optimizer_time = result.optimizer_time
print("Optimal parameters (optimal_point):", optimal_point)
print("Optimal eigenvalue (optimal_value):", optimal_value)
print("Number of optimizer evaluations (optimizer_evals):", optimizer_evals)
print("Optimizer result:", optimizer_result)
print("Time taken by optimizer (optimizer_time):", optimizer_time)

Optimal parameters (optimal_point): [-5.46769504  3.87095598  2.92356953  1.8147622 ]
Optimal eigenvalue (optimal_value): 3.3306690738754696e-16
Number of optimizer evaluations (optimizer_evals): None
Optimizer result: {   'fun': np.float64(3.3306690738754696e-16),
    'jac': None,
    'nfev': 25,
    'nit': 3,
    'njev': 5,
    'x': array([-5.46769504,  3.87095598,  2.92356953,  1.8147622 ])}
Time taken by optimizer (optimizer_time): 0.07225608825683594


In [ ]:
min_eigenvalue = result.eigenvalue
print("Minimum eigenvalue:", min_eigenvalue)

Minimum eigenvalue: 3.3306690738754696e-16


In [10]:
import qiskit_aer
print(qiskit_aer.__version__)

0.15.1


In [12]:
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit_algorithms import VQE
from qiskit.primitives import Estimator

In [14]:
dir(NoiseModel)

['_1qubit_instructions',
 '_2qubit_instructions',
 '_3qubit_instructions',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_all_qubit_quantum_errors_equal',
 '_check_number_of_qubits',
 '_instruction_names_labels',
 '_keys2str',
 '_local_quantum_errors_equal',
 '_pass_manager',
 '_readout_errors_equal',
 'add_all_qubit_quantum_error',
 'add_all_qubit_readout_error',
 'add_basis_gates',
 'add_quantum_error',
 'add_readout_error',
 'basis_gates',
 'from_backend',
 'from_backend_properties',
 'from_dict',
 'is_ideal',
 'noise_instructions',
 'noise_qubits',
 'reset',
 'to_dict']

In [ ]:
noise_model = NoiseModel()
depolarizing_1q = depolarizing_error(0.01, 1) 
depolarizing_2q = depolarizing_error(0.01, 2) 
noise_model.add_quantum_error(depolarizing_1q, ['u3'], [0]) 
noise_model.add_quantum_error(depolarizing_1q, ['u3'], [1]) 
noise_model.add_quantum_error(depolarizing_2q, ['cx'], [0, 1]) 
simulator_with_noise = AerSimulator(noise_model=noise_model)
def zero_noise_extrapolation(hamiltonian, var_form, optimizer, noise_level):
    noise_model = NoiseModel()
    noise_model.add_quantum_error(depolarizing_error(noise_level, 1), ['u3'], [0])
    noise_model.add_quantum_error(depolarizing_error(noise_level, 1), ['u3'], [1])
    noise_model.add_quantum_error(depolarizing_error(noise_level, 2), ['cx'], [0, 1])
    simulator = AerSimulator(noise_model=noise_model)
    vqe = VQE(estimator=Estimator(), ansatz=var_form, optimizer=optimizer)
    result = vqe.compute_minimum_eigenvalue(hamiltonian)    
    return result.eigenvalue
eigenvalues = []
for noise_level in [0.01, 0.05, 0.1]:
    eigenvalues.append(zero_noise_extrapolation(hamiltonian, var_form, optimizer, noise_level))
print("Eigenvalues for different noise levels:", eigenvalues)

C:\Users\MEGANATH N\AppData\Local\Temp\ipykernel_15384\2913986126.py:35: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe = VQE(estimator=Estimator(), ansatz=var_form, optimizer=optimizer)
C:\Users\MEGANATH N\AppData\Local\Temp\ipykernel_15384\2913986126.py:35: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe = VQE(estimator=Estimator(), ansatz=var_form, optimizer=optimizer

Eigenvalues for different noise levels: [np.float64(0.0), np.float64(1.4432899320127035e-15), np.float64(0.0)]


In [ ]:
from qiskit_aer import AerSimulator, NoiseModel
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import StatevectorEstimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_aer.noise import depolarizing_error
var_form = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz', reps=3)
hamiltonian = SparsePauliOp.from_list([("Z", 1), ("I", 1)])
optimizer = COBYLA(maxiter=100)
def zero_noise_extrapolation(hamiltonian, var_form, optimizer, noise_level):
    noise_model = NoiseModel()
    noise_model.add_quantum_error(depolarizing_error(noise_level, 1), ['u3'], [0]) 
    noise_model.add_quantum_error(depolarizing_error(noise_level, 1), ['u3'], [1])  
    noise_model.add_quantum_error(depolarizing_error(noise_level, 2), ['cx'], [0, 1]) 
    simulator = AerSimulator(noise_model=noise_model)
    vqe = VQE(estimator=StatevectorEstimator(), ansatz=var_form, optimizer=optimizer)
    result = vqe.compute_minimum_eigenvalue(hamiltonian)
    return result.eigenvalue.real
eigenvalues = []
for noise_level in [0.01, 0.05, 0.1]:
    eigenvalues.append(zero_noise_extrapolation(hamiltonian, var_form, optimizer, noise_level))
print("Eigenvalues for different noise levels:", [float(ev) for ev in eigenvalues])


Eigenvalues for different noise levels:
Noise level 1: 0.0
Noise level 2: 1.4432899320127035e-15
Noise level 3: 0.0


In [ ]:
optimal_eigenvalue = 3.3306690738754696e-16  
ideal_eigenvalue = 0.0
error_before_zne = abs(ideal_eigenvalue - optimal_eigenvalue)
eigenvalues = []
for noise_level in [0.09, 0.05, 0.07]:
    eigenvalues.append(zero_noise_extrapolation(hamiltonian, var_form, optimizer, noise_level))
noise_levels = np.array([0.09, 0.05, 0.07])
noisy_eigenvalues = np.array(eigenvalues)
slope, intercept = np.polyfit(noise_levels, noisy_eigenvalues, 1)
zne_eigenvalue = intercept  
error_after_zne = abs(ideal_eigenvalue - zne_eigenvalue)
print("Optimal eigenvalue (from VQE):", optimal_eigenvalue)
print("Error before ZNE:", error_before_zne)
print("\nEigenvalues for different noise levels (before ZNE):")
for i, eigenvalue in enumerate(eigenvalues):
    print(f"Noise level {noise_levels[i]}: {eigenvalue}")
print("\nExtrapolated eigenvalue after ZNE (zero noise):", zne_eigenvalue)
print(f"Error after ZNE: {error_after_zne}")

Optimal eigenvalue (from VQE): 3.3306690738754696e-16
Error before ZNE: 3.3306690738754696e-16

Eigenvalues for different noise levels (before ZNE):
Noise level 0.09: 1.0955680807001045e-12
Noise level 0.05: 3.774758283725532e-15
Noise level 0.07: 1.1102230246251565e-16

Extrapolated eigenvalue after ZNE (zero noise): -1.5441536937999006e-12
Error after ZNE: 1.5441536937999006e-12


C:\Users\MEGANATH N\AppData\Local\Temp\ipykernel_15384\2913986126.py:35: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe = VQE(estimator=Estimator(), ansatz=var_form, optimizer=optimizer)
C:\Users\MEGANATH N\AppData\Local\Temp\ipykernel_15384\2913986126.py:35: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe = VQE(estimator=Estimator(), ansatz=var_form, optimizer=optimizer

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
quantum_eigenvalues = np.array([float(eigenvalue) for eigenvalue in eigenvalues])
normalized_eigenvalues = (quantum_eigenvalues - np.min(quantum_eigenvalues)) / (np.max(quantum_eigenvalues) - np.min(quantum_eigenvalues))
similarity_matrix = cosine_similarity(normalized_eigenvalues.reshape(-1, 1))
from sklearn.metrics.pairwise import rbf_kernel
similarity_matrix = rbf_kernel(normalized_eigenvalues.reshape(-1, 1), gamma=0.1)

In [ ]:
from sklearn.cluster import AffinityPropagation
affinity_model = AffinityPropagation(preference=-0.5)  
affinity_model.fit(similarity_matrix)
quantum_cluster_labels = affinity_model.labels_
print("Quantum-enhanced cluster labels:", quantum_cluster_labels)
n_clusters = len(set(quantum_cluster_labels))
print(f"Number of clusters found: {n_clusters}")

Quantum-enhanced cluster labels: [0 0 0]
Number of clusters found: 1


In [ ]:
from sklearn.cluster import AffinityPropagation
import numpy as np
quantum_eigenvalues = np.array([1.0955680807001045e-12, 3.774758283725532e-15, 1.1102230246251565e-16])
normalized_eigenvalues = (quantum_eigenvalues - np.min(quantum_eigenvalues)) / (np.max(quantum_eigenvalues) - np.min(quantum_eigenvalues))
similarity_matrix = np.exp(-np.square(normalized_eigenvalues))
affinity_model = AffinityPropagation(preference=-50) 
affinity_model.fit(similarity_matrix.reshape(-1, 1)) 
quantum_cluster_labels = affinity_model.labels_
print("Quantum-enhanced cluster labels:", quantum_cluster_labels)

Quantum-enhanced cluster labels: [0 0 0]


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
df = pd.read_csv(r"D:\QUANT A MAZE\cancerdata1.csv")
df = df.drop(columns=['id']) 
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})
features = df.drop(columns=['diagnosis'])
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)
scaled_data = pd.DataFrame(scaled_features, columns=features.columns)
scaled_data['diagnosis'] = df['diagnosis']
print(scaled_data.head())

   radius_mean  texture_mean  perimeter_mean  area_mean  smoothness_mean  \
0     1.730303     -1.469461        1.724207   1.932045        -0.103481   
1    -0.652430     -1.102501       -0.573276  -0.687675         2.098192   
2     1.110306      0.052346        1.046290   1.110887        -0.557426   
3    -0.269491      0.281696       -0.239566  -0.365601         1.385650   
4    -0.485274      0.548822       -0.357655  -0.551241         2.058161   

   compactness_mean  concavity_mean  concave points_mean  symmetry_mean  \
0          0.158779        1.135764             1.146484      -0.363668   
1          0.801843        0.610292             0.514924       0.556388   
2         -0.252644        0.020771             0.329044      -0.413311   
3          0.706766       -0.228109            -0.052698       0.917129   
4          1.202894        0.977599             0.855928       1.426801   

   fractal_dimension_mean  ...  texture_se  perimeter_se   area_se  \
0               -0.739

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3) 
kmeans.fit(scaled_data.drop(columns=['diagnosis']))
classical_cluster_labels = kmeans.labels_
print("Classical KMeans cluster labels:", classical_cluster_labels)

Classical KMeans cluster labels: [0 1 0 1 1 2 1 0 2 0 2 0 1 0 0 1 1 1 2 0 0 2 1 0 0 0 0 1 0 0 1 0 1 1 0 1 1
 1 2 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 2 1 1 1 1 1 2 1 0 2 0 1 1 0 1 2
 2 1 1 1 2 2 1 0 1 0 1 1 1 1 1 1 0 0 1 1 1]


In [ ]:
print(len(quantum_cluster_labels), len(classical_cluster_labels))

10 95


In [102]:
from sklearn.metrics import adjusted_rand_score
ari_score = adjusted_rand_score(quantum_cluster_labels, classical_cluster_labels)
print(f"Adjusted Rand Index (ARI): {ari_score:.2f}")

Adjusted Rand Index (ARI): 0.00


In [103]:
from sklearn.metrics import normalized_mutual_info_score
nmi_score = normalized_mutual_info_score(quantum_cluster_labels, classical_cluster_labels)
print(f"Normalized Mutual Information (NMI): {nmi_score:.2f}")

Normalized Mutual Information (NMI): 0.00


In [104]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(quantum_cluster_labels, classical_cluster_labels)
print(f"Accuracy between Quantum and Classical Clustering: {accuracy * 100:.2f}%")

Accuracy between Quantum and Classical Clustering: 40.00%
